# 4차시
## 자동 웹크롤링
#### Selenium을 활용하여 자동 웹크롤링 코드를 작성할 것이다.
#### 오늘 배울 것은 코드을 이해를 하고 다음번에 다른 사이트의 크롤링을 사용할 수 있는 방법들을 얘기하겠다.
#### 총 3가지의 코드를 보여줄 것이다.
#### 1. 유튜브 영상 댓글 가져오기
#### 2. 구글 이미지 가져오기
#### 3. 국민 신문고 데이터 가져오기

#### 아래의 코드들을 다 이해해라는 말은 하지 않겠습니다. 흐름을 이해하고 어떤 식으로 구성되는지를 아시면 됩니다.

In [2]:
'''Step 1. 필요한 모듈과 라이브러리를 로딩합니다.'''

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import openpyxl


'''Step 2. 사용자에게 검색어 키워드를 입력 받습니다.'''

print("=" *80)
print("연습문제 : 유투브 영상의 댓글 수집하기")
print("=" *80)

'''input 함수를 사용하여 컴퓨터와 상호작용을 합니다.'''

query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")
cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))
f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")



'''저장될 파일위치와 이름을 지정합니다'''
'''현재시간을 받아줍니다.'''

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)


'''저장할 폴더를 만들어줍니다.'''

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)


'''저장할 파일명과 경로를 작성해줍니다.'''

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

'''얼마나 걸렸는지 시간을 보기위해 실행시킵니다.'''

s_time = time.time( )


'''Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.'''

path = "C:\\Users\\a3011\\Downloads\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://www.youtube.com')

'''아래 코드는 5초뒤에 아래코드를 시작한다는 것입니다.'''

time.sleep(5)

element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

'''
검색 결과가 첫 화면에 20개가 나옵니다.
만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다
'''

'''유튜브 댓글은 스크롤을 내려야 더 추가가 되기 때문에 스크롤 내리는 코드를 작성해줍니다.'''

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

'''유튜브 댓글은 한번에 20개를 보여주며, 스크롤을 내릴때마다 20개씩 추가로 보여줍니다.'''
'''그래서 댓글을 긁어오는 양에 따라 스크롤을 몇번 내려줄지 정하는 코드입니다.'''

if reple_cnt < 20 :
        page_cnt = 1 
else :
        page_cnt = math.ceil(cnt/20)
        
if cnt > 20 :
           
    i = 1
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1


'''검색된 유튜브 영상의 URL 을 추출합니다'''

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
          
    item.append(i['href'])
    count += 1

    if count == cnt :
        break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")


'''비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다'''
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)


'''
수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
'''

full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.youtube.com/' +item[x]
    full_url.append(url)
    if url_cnt == cnt :
        break


play_cnt = 1

'''파싱한 데이터들을 저장할 공간을 지정해 줍니다.'''


url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]


'''엑셀 형태로 저장하기 위해 실행하는 코드입니다.'''
wb=openpyxl.Workbook( )
wb.save(fx_name)

''' 동영상의 기본정보를 추출하는 코드입니다.'''


for y in range(0,len(full_url)) :
    driver.get(full_url[y])
    time.sleep(10)
    print("\n")
    print("\n")
    print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

    i = 1
    while ( i <=  page_cnt) :
    #print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(10)
        i += 1
      
    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')
        
    t_count_0 = soup2.select('#count')
    t_count_1 = t_count_0[1].get_text()
    t_count_2 = t_count_1.replace(",","")
    t_count_3 = re.search("\d+",t_count_2)
    t_count_4 = int(t_count_3.group())

    t_title_1 = soup2.select('#info-contents')
    t_title_2 = t_title_1[0].find('h1').get_text()
    t_title_3 = t_title_2.translate(bmp_map).replace("\n","")

    t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()
    t_view_2 = t_view_1.replace("," ,"")
    t_view_3 = re.search("\d+",t_view_2)
    print(t_view_1)
    t_view_4 = int(t_view_3.group())

    print("=" *80)
    print("%s 번째 동영상의 조회수는 %s 회 이고 수집할 댓글은 총 %s개 입니다" %(play_cnt,t_count_4,t_view_4))  
    print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
    print("=" *80)
    print("\n")
    print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
    print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
    print("\n")

'''Step 5. 화면을 스크롤해서 아래로 이동합니다'''
    i = 1
    while ( i <=  page_cnt+1) :
        print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
        scroll_down(driver)
        time.sleep(0.5)
        i += 1

'''Step 6. 내용을 수집합니다'''

    time.sleep(2)

    html = driver.page_source
    soup3 = BeautifulSoup(html, 'html.parser')
        
    count = 0
    d2 = 0

    reple_result = soup3.select('#comments > #sections > #contents')

    for a in range(0,reple_cnt+1) :
        count += 1
              
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("\n")
        f.write("------------------------------------------------------------------"+"\n")

             # 댓글 작성자
        try : 
            writer = reple_result[0].select("#author-text > span")[a].get_text().replace("\n","").strip()
        except IndexError :
            print('error')
            break
        else :
            print("\n")
            print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
            print("-" *70)

                # 유튜브 URL 주소
            print("1.URL 주소:",full_url[y])
            f.write("1.유투브 URL주소: " + full_url[y] + "\n")
            url2.append(full_url[y])
                
            print("2.댓글 작성자명:", writer)
            f.write("2.댓글 작성자명: " + writer + "\n")
            writer2.append(writer)
              
            # 댓글 작성일자
            wdate = reple_result[0].select("yt-formatted-string.published-time-text > a.yt-simple-endpoint")[a].get_text().replace("\n","")
            
            print("3.댓글 작성일자:",wdate)
            f.write("3.댓글 작성일자: " + wdate + "\n")
            wdate2.append(wdate)
              
            # 댓글 내용
            reple1 = reple_result[0].select('#expander > #content > #content-text')[a].get_text().replace("\n","")
            
            reple2=reple1.translate(bmp_map).replace("\n","")
            print("4.댓글 내용:",reple2)
            f.write("4.댓글 내용: " + reple2 + "\n")
            reple3.append(reple2)
                
              
            f.close( )
              
            if count == reple_cnt :
                break
              
    time.sleep(2)       
    play_cnt += 1                

'''step 9. 엑셀 형태로 저장하기'''
    
              
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )


print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

time.sleep(3)

driver.close( )

 

유투브 영상의 댓글 수집하기


1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영):  올리브영
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?: 10
3.각 동영상에서 추출할 댓글은 몇건입니까?:  10
4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\temp\): C:\Users\a3011\OneDrive\Desktop\유튜브댓글모음




요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^


검색된 영상 중 10 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.
댓글 8개
1 번째 동영상의 조회수는 322 회 이고 수집할 댓글은 총 8개 입니다
1 번째 동영상의 제목은 �집에서 쉽게 케어하자! 목주름 미리 관리하기! [뷰티위키] l 올리브영(Oliveyoung) l 소의튜브 입니다


1 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~




3 번째 동영상의 정보를 수집합니다.
댓글 11개
3 번째 동영상의 조회수는 92928 회 이고 수집할 댓글은 총 11개 입니다
3 번째 동영상의 제목은 피부과 직원 10년차! 소의튜브가 알려주는 민감피부 케어 방법! [뷰티위키] l 올리브영(Oliveyoung) l 소의튜브 입니다


3 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~




5 번째 동영상의 정보를 수집합니다.
댓글 351개
5 번째 동영상의 조회수는 330674 회 이고 수집할 댓글은 총 351개 입니다
5 번째 동영상의 제목은 올리브영에서 재구매한 제품들! (겨관리/피부좋아지는 바디,스킨케어/머릿결관리/메이크업정착템) | 민스코 Minsco 입니다


5 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~




7 번째 동영상의 정보를 수집합니다.
댓글 129개
7 번째 동영상의 조회수는 15062 회 이고 수집할 댓글은 총 129개 입니다
7 번째 동영상의 제목은 반갑습니다 

<ipython-input-2-f2a5e052a2d1>:255: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  youtube_reple['댓글작성자명']=pd.Series(writer2)
<ipython-input-2-f2a5e052a2d1>:256: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  youtube_reple['댓글작성일자']=pd.Series(wdate2)
<ipython-input-2-f2a5e052a2d1>:257: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  youtube_reple['댓글 내용']=pd.Series(reple3)




1.요청된 총 10 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 1 건입니다
2.총 소요시간은 506.2 초 입니다 
3.파일 저장 완료: txt 파일명 : C:\Users\a3011\OneDrive\Desktop\유튜브댓글모음2021-03-25-09-07-50-올리브영\2021-03-25-09-07-50-올리브영.txt 
4.파일 저장 완료: csv 파일명 : C:\Users\a3011\OneDrive\Desktop\유튜브댓글모음2021-03-25-09-07-50-올리브영\2021-03-25-09-07-50-올리브영.csv 
5.파일 저장 완료: xls 파일명 : C:\Users\a3011\OneDrive\Desktop\유튜브댓글모음2021-03-25-09-07-50-올리브영\2021-03-25-09-07-50-올리브영.xls 


In [ ]:
'''구글 이미지 수집용 크롤러'''

'''Step 1. 필요한 모듈과 라이브러리를 로딩합니다.'''

from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib
import time
import sys

import math
import time
import os
import random

'''Step 2. 필요한 정보를 입력 받습니다.'''


print("=" *80)
print("구글 사이트에서 이미지를 검색하여 수집하는 크롤러 입니다 ")
print("=" *80)

query_txt = input('1.크롤링할 이미지의 키워드는 무엇입니까?: ')
cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))

real_cnt = math.ceil(cnt / 50) # 실제 크롤링 할 페이지 수

f_dir=input('3.파일이 저장될 경로만 쓰세요(예: c:\\temp\\ ) : ')

if f_dir =='' :
    f_dir = "c:\\temp\\"
print("\n")
print("요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~^^")

#Step 3. 파일을 저장할 폴더를 생성합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.chdir(f_dir)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
f_result_dir = f_dir+s+'-'+query_txt
                

'''Step 4. 크롬 드라이버를 사용해서 웹 브라우저를 실행한 후 검색합니다'''


s_time = time.time( )

path = "C:\\Users\\a3011\\Downloads\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://www.google.com')
time.sleep(random.randrange(2,5))

element = driver.find_element_by_name("q")

element.send_keys(query_txt)
element.submit()

'''Step 5. 아래의 이미지 링크를 선택합니다'''

driver.find_element_by_link_text("이미지").click()
        
'''스크롤 다운 함수 만들기'''

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)

i = 1
while (i <= real_cnt):
    scroll_down(driver) 
    i += 1

    if i ==  6 :
        driver.find_element_by_xpath("""//*[@id="_sau_imageTab"]/div[2]/div[8]/a""").click()

'''Step 6. 이미지 추출하여 저장하기 '''

file_no = 0
count = 1
img_src2=[]

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

imgs = driver.find_elements_by_tag_name('img')

for img in imgs:
        img_src1=img.get_attribute('src')
        img_src2.append(img_src1)
        count += 1

for i in range(2,len(img_src2)+1) :

        try :
                urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
        except TypeError:
                continue
        
        file_no += 1
                
        time.sleep(1)
        print("\n")
        print("%s 번째 이미지 저장중입니다=======" %file_no)
        print("\n")

        if file_no == cnt :
                break

'''Step 7. 요약 정보를 출력합니다'''
e_time = time.time( )
t_time = e_time - s_time

store_cnt = file_no -1

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %f_result_dir)
print("=" *70)

driver.close( )

In [ ]:
'''청와대 국민 신문고 추천순 데이터 수집하기'''

'''Step 1. 필요한 모듈과 라이브러리를 로딩합니다.'''

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os   

'''Step 2. 사용자에게 검색어 키워드를 입력 받습니다.'''
print("=" *80)
print(" 연습문제: 청와대 국민 신문고 게시판 크롤링하기")
print("=" *80)

query_txt = '청와대국민신문고'

cnt = int(input("   1.크롤링 할 건수는 몇건입니까?: "))
f_dir = input("   2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

'''저장될 파일위치와 이름을 지정합니다'''
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


'''Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.'''
s_time = time.time( )

path = "C:\\Users\\a3011\\Downloads\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://www1.president.go.kr/petitions/best')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택


print("\n")
page_cnt = math.ceil(cnt / 15)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

'''Step 4.  게시글 요약 정보 출력'''

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

click_cnt = 2   # 클릭한 페이지 번호 지정
no = 1
cno2=[]
title2=[]
part2=[]
rdate2=[]
status2=[]
view2=[]

for x in range(1, page_cnt+1) :
        
        print("%s 페이지의 요약 내용 수집 시작합니다 =======================" %x)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        result = soup.select('#cont_view > div.cs_area > div > div > div.board.text')
        
        for i in range(0, len(result[0].find_all('li')) ):  
                
            print("\n")
            print("%s 번째 국민 신문고 게시글 정보입니다==========================" %no)
            
            f = open(ff_name, 'a',encoding='UTF-8')
            
            # 청원 등록 정보 수집
            content = result[0].find_all('li')
            
            cno = content[i].find(class_='bl_no').text.split(' ')[-1]   
            print('1.청원글번호: ',cno ,'\n')
            f.write('\n')
            f.write("%s 번째 국민 신문고 게시글 정보입니다==========================" %no + '\n')
            f.write('1.청원번호:' + cno + '\n')
            cno2.append(cno)
            
            # 분류 추출
            title = content[i].find(class_='bl_category cs').text.split(' ')[-1]
            print('2.분류: ' , title , '\n')
            f.write('2.분류:' + title + '\n')
            title2.append(title)
            
            # 청원만료일 추출
            rdate = content[i].find(class_='bl_date light').text
            print('3.청원만료일: ' , rdate , '\n')
            f.write('3.청원만료일:' + rdate + '\n')
            rdate2.append(rdate)            
            
            # 제목 추출
            title = content[i].find(class_='cb').text[3:]
            print('4.제목: ' , title , '\n')
            f.write('4.제목:' + title + '\n')
            title2.append(title)
            
            # 참여인원 추출
            view = content[i].find(class_='bl_agree cb wv_agree').text.split()[-1]
            print('5.참여인원: ' , view , '\n')
            f.write('5.참여인원:' + view + '\n')
            view2.append(view)
            
            f.close( )
            
            no += 1
            
            if no > cnt :
                break
                
        if click_cnt > page_cnt :
                break
        else :
                driver.find_element_by_link_text("""%s""" %click_cnt).click() # 다음 페이지번호 클릭
                click_cnt += 1
            
                time.sleep(2)
'''
Step 5. 출력 결과를 저장합니다
출력 결과를 표(데이터 프레임) 형태로 만들기
'''
sin = pd.DataFrame()

sin['글번호'] = cno2
sin['글제목'] = pd.Series(title2) 
sin['청원만료일'] = pd.Series(rdate2)
sin['참여인원'] = pd.Series(view2)
            
'''csv 형태로 저장하기'''
sin.to_csv(fc_name, encoding="utf-8-sig" , index=False)

'''엑셀 형태로 저장하기'''
sin.to_excel(fx_name,index=False)

'''Step 6. 요약 정보 출력하기'''
e_time = time.time( )     
t_time = e_time - s_time
print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

#### 총 3가지의 웹페이지를 파싱했습니다.
#### 논문이나 리포터를 작성할 때, 이렇게 데이터를 모아서 사용하시면 됩니다.
#### 다음 차시에는 모은 데이터를 어떻게 분석 또는 시각화를 활용하여 의미있는 데이터를 만드는 과정을 합니다.